# Cálculo de volumenes y parámetros de las capas de una red neuronal

Crea un fichero yaml describiendo las capas:

```
---

- name: 'my awesome cnn'
  layers:
    - type: 'Conv2D'
      input:
        - 32  # width
        - 32  # height
        - 3   # depth
      filters:
        - 5  # size
        - 10 # number
      stride: 1
      padding: 2
    - type: 'MaxPooling2D'
      input:
        - 32  # width
        - 32  # height
        - 3   # depth
      filters:
        - 5  # size
      stride: 1
    - type: 'Dense'
      input: 10   # previous layer neurons
      neurons: 15 # neurons in fully connected layer
```

In [57]:
import yaml
import sys

# cuatro hiperparámetros
#  * Tamaño de filtro o pool F
#  * Stride S
#  * Número de filtros K
#  * Cantidad de 0-padding P
#
# parámetros
#  * F * F * D1 pesos por filtro
#  * 1 bias por filtro
#
# Volumen de entrada W1 x H1 x D1
#
# Volumen de salida W2 x H2 x D2
#
# W2 = (W1 - F + 2*P)/S + 1
# H2 = (H1 -F + 2*P)/S + 1
# D2 = K

def conv2d(layer):
    filter_size = layer['filters'][0]
    filter_number = layer['filters'][1]
    padding = layer['padding']
    stride = layer['stride']
    w = ((layer['input'][0] - filter_size + (2*padding))/stride) + 1
    h = ((layer['input'][1] - filter_size + (2*padding))/stride) + 1
    return [w, h, filter_number]

# dos hiperparámetros
#  * Tamaño de filtro o pool F
#  * Stride S
#
# no introduce parámetros
#
# Volumen de entrada W1 x H1 x D1
#
# Volumen de salida W2 x H2 x D2
#
# W2 = (W1 - F)/S + 1
# H2 = (H1 -F)/S + 1
# D2 = D1

def maxpooling2d(layer):
    filter_size = layer['filters'][0]
    stride = layer['stride']
    w = ((layer['input'][0] - filter_size)/stride) + 1
    h = ((layer['input'][1] - filter_size)/stride) + 1
    return [w, h, layer['input'][2]]

# fully connected
# (previous layer neurons * current layer neurons) + current layer neurons
# we add the current layer neurons due to the bias

def dense(layer):
    return ((layer['input'] * layer['neurons']) + layer['neurons'])

options = {'Conv2D': conv2d,
           'MaxPooling2D': maxpooling2d,
           'Dense': dense
        }

In [58]:
import pyaml

yaml_file = 'cnn-test.yaml'
output_file = 'cnn-test-output.yaml'

try:
    with open(yaml_file, 'r') as file:
        description = yaml.load(file, Loader=yaml.FullLoader)

except yaml.YAMLError as exc:
    print(exc)
    sys.exit(1)

for item in description[0]['layers']:
    # obtenemos el input shape
    if item['type'] == 'Dense':
        output = options[item['type']](item)
        item['parameters'] = output
    elif item['type'] == 'Conv2D':
        output = options[item['type']](item)
        item['output_volume'] = output
        item['parameters'] = {}
        item['parameters']['filter'] = (item['filters'][0] * item['filters'][0] * item['input'][2]) + 1
        item['parameters']['total'] = item['parameters']['filter'] * item['output_volume'][2]
    elif item['type'] == 'MaxPooling2D':
        output = options[item['type']](item)
        item['output_volume'] = output

with open(output_file, 'w') as outfile:
    yaml.dump(description, outfile, default_flow_style=False)
    
print(pyaml.dump(description))

- layers:
    - filters:
        - 5
        - 10
      input:
        - 32
        - 32
        - 3
      output_volume:
        - 32.0
        - 32.0
        - 10
      padding: 2
      parameters:
        filter: 76
        total: 760
      stride: 1
      type: Conv2D
    - filters:
        - 5
      input:
        - 32
        - 32
        - 3
      output_volume:
        - 28.0
        - 28.0
        - 3
      stride: 1
      type: MaxPooling2D
    - input: 10
      neurons: 15
      parameters: 165
      type: Dense
  name: 'my awesome cnn'

